# Выбор локации для скважины

Нефтедобывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. А также проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

### Первичный анализ данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

display(data_0.head(3))
display(data_1.head(3))
display(data_2.head(3))

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


In [3]:
data_0.info()
data_1.info()
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

Данные в 3 датасетах корректных типов, не содержат пропусков и не нуждаются в предобработке.

### Подготовка данных для обучения

In [4]:
features_0 = data_0.drop(['id', 'product'], axis=1)
target_0 = data_0['product']

features_1 = data_1.drop(['id', 'product'], axis=1)
target_1 = data_1['product']

features_2 = data_2.drop(['id', 'product'], axis=1)
target_2 = data_2['product']

In [5]:
#разделение на обучающую и валидационную выборки
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345)

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

In [6]:
print(features_train_0.shape)
print(features_valid_0.shape)
print(target_train_0.shape)
print(target_valid_0.shape)

print(features_train_1.shape)
print(features_valid_1.shape)
print(target_train_1.shape)
print(target_valid_1.shape)

print(features_train_2.shape)
print(features_valid_2.shape)
print(target_train_2.shape)
print(target_valid_2.shape)

(75000, 3)
(25000, 3)
(75000,)
(25000,)
(75000, 3)
(25000, 3)
(75000,)
(25000,)
(75000, 3)
(25000, 3)
(75000,)
(25000,)


## Обучение и проверка модели

In [7]:
#функция для обучения моделей и вывода ключевых метрик
def lr_learning(features_train, target_train, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = pd.Series(model.predict(features_valid))

    mse = mean_squared_error(target_valid, predictions)
    rmse = mse ** 0.5

    print('Средний реальный запас сырья', target_valid.mean())
    print('Средний предсказанный запас сырья', predictions.mean())
    print('RMSE:', rmse)

    predictions_dummy = pd.Series(target_valid.mean(), index=target_valid.index)
    mse = mean_squared_error(target_valid, predictions_dummy)
    rmse = mse ** 0.5

    print('RMSE при заполнении средним:', rmse)
    
#вспомогательная функция для вывода предсказаний
def predicts(features_train, target_train, features_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = pd.Series(model.predict(features_valid))
    return predictions

### Регион 1

In [8]:
lr_learning(features_train_0, target_train_0, features_valid_0, target_valid_0)
predictions_0 = predicts(features_train_0, target_train_0, features_valid_0)

Средний реальный запас сырья 92.07859674082927
Средний предсказанный запас сырья 92.59256778438035
RMSE: 37.5794217150813
RMSE при заполнении средним: 44.28602687855358


При средних запасах сырья 92.078 средняя RMSE=37.579 - это сильное отклонение, хотя этот результат значительно лучше, чем заполненный средним значением целевого показателя = 44.286

In [9]:
#попробуем то же самое, но с масштабированием
scaler = StandardScaler()
scaler.fit(features_train_0)

features_train_scaled = scaler.transform(features_train_0)
features_valid_scaled = scaler.transform(features_valid_0)

model_01 = LinearRegression()
model_01.fit(features_train_scaled, target_train_0)
predictions_01 = pd.Series(model_01.predict(features_valid_scaled))

mse_01 = mean_squared_error(target_valid_0, predictions_01)
rmse_01 = mse_01 ** 0.5

print('Средний реальный запас сырья', target_valid_0.mean())
print('Средний предсказанный запас сырья', predictions_01.mean())
print('RMSE:', rmse_01)

pd.options.mode.chained_assignment = None

Средний реальный запас сырья 92.07859674082927
Средний предсказанный запас сырья 92.59256778438035
RMSE: 37.5794217150813


Масштабирование признаков не улучшило RMSE модели. Возможно, признаки изначально в одном масштабе

In [10]:
data_0['predict_product'] = pd.Series(predictions_01, index=target_valid_0.index)
data_0['predict_dummy'] = pd.Series(target_valid_0.mean(), index=target_valid_0.index)
display(data_0.head(10))

,id,f0,f1,f2,product,predict_product,predict_dummy
0,txEyH,0.705745,-0.497823,1.221170,105.280062,NaN,NaN
1,2acmU,1.334711,-0.340164,4.365080,73.037750,77.572583,92.078597
2,409Wp,1.022732,0.151990,1.419926,85.265647,77.892640,92.078597
3,iJLyR,-0.032172,0.139033,2.978566,168.620776,90.175134,92.078597
4,Xdl7t,1.988431,0.155413,4.751769,154.036647,NaN,NaN
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541,NaN,NaN
6,tL6pL,0.645075,0.530656,1.780266,49.055285,NaN,NaN
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292,NaN,NaN
8,j9Oui,0.643105,-0.551583,2.372141,113.356160,86.035871,92.078597
9,OLuZU,2.173381,0.563698,9.441852,127.910945,NaN,NaN


### Регион 2 

In [11]:
lr_learning(features_train_1, target_train_1, features_valid_1, target_valid_1)
predictions_1 = predicts(features_train_1, target_train_1, features_valid_1)

Средний реальный запас сырья 68.72313602435997
Средний предсказанный запас сырья 68.728546895446
RMSE: 0.893099286775617
RMSE при заполнении средним: 46.0212449226281


С данными по 2 региону модель отработала намного лучше - видимо, связь между признаками более линейная, чем в данных по региону 1. RMSE=0.89 - хороший результат

### Регион 3 

In [12]:
lr_learning(features_train_2, target_train_2, features_valid_2, target_valid_2)
predictions_2 = predicts(features_train_2, target_train_2, features_valid_2)

Средний реальный запас сырья 94.88423280885438
Средний предсказанный запас сырья 94.96504596800489
RMSE: 40.02970873393434
RMSE при заполнении средним: 44.902084376898294


Итог, близкий к данным первого региона - у модели высокая RMSE=40.029, немногим больше чем при заполнении средним.

## Подготовка к расчёту прибыли

In [13]:
# Бюджет на разработку скважин в регионе
region_budget = 10000000000

# Доход с каждой тысячи баррелей
revenue_1000_barrel = 450000

# достаточный объём сырья в тысячах баррелей для безубыточной разработки новой скважины
profitable_value = region_budget / revenue_1000_barrel
print('Объём сырья в тысячах баррелей для безубыточной разработки:', round(profitable_value))
print()
print('Средний запас 200 скважин в регионе 1:', round(target_valid_0.mean() * 200))
print('Средний запас 200 скважин в регионе 2:', round(target_valid_1.mean() * 200))
print('Средний запас 200 скважин в регионе 3:', round(target_valid_2.mean() * 200))

Объём сырья в тысячах баррелей для безубыточной разработки: 22222

Средний запас 200 скважин в регионе 1: 18416
Средний запас 200 скважин в регионе 2: 13745
Средний запас 200 скважин в регионе 3: 18977


При расчете от средних запасов получается, что все 3 региона потенциально убыточны. Однако, такой подход не совсем корректен, т.к. он подразумевает отбор скважин случайным образом вместо того, чтобы отобрать лучшие 200 скважин в каждом регионе.

In [14]:
#функция для расчета прибыли
def profit(target, predictions, count):
    best_pred = pd.Series(predictions).sort_values(ascending=False)[:count]
    best_target = target[best_pred.index]
    profit = revenue_1000_barrel * best_target.sum() - region_budget
    return profit

## Расчёт прибыли и рисков 

In [15]:
profit_0 = profit(target_0, predictions_0, 200)
print('Потенциальная прибыль региона 1, млн руб:', round(profit_0 / 1000000))

profit_1 = profit(target_1, predictions_1, 200)
print('Потенциальная прибыль региона 2, млн руб:', round(profit_1 / 1000000))

profit_2 = profit(target_2, predictions_2, 200)
print('Потенциальная прибыль региона 3, млн руб:', round(profit_2 / 1000000))

Потенциальная прибыль региона 1, млн руб: -1922
Потенциальная прибыль региона 2, млн руб: -4016
Потенциальная прибыль региона 3, млн руб: -1259


При таком подходе подтверждается вывод с предыдущего этапа - все 3 региона потенциально убыточны. Применим технику bootstrap

### Поиск самого прибыльного региона техникой bootstrap

In [16]:
#временная ячейка для проверки размера best_target во время бутстрапирования
state = np.random.RandomState(12345)

def region_profit(target, predictions):
    values = []
    for i in range(1000):
        target_subsample = target.reset_index(drop=True).sample(500, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]
        best_pred = pd.Series(probs_subsample).sort_values(ascending=False)[:200]
        best_target = target_subsample[best_pred.index]
        revenues = revenue_1000_barrel * best_target.sum() - region_budget
        values.append(revenues)
    
    return best_target.shape, best_pred.shape

region_profit(target_valid_0, predictions_0)

((206,), (200,))

In [17]:
state = np.random.RandomState(12345)

def region_profit(target, predictions):
    values = []
    for i in range(1000):
        target_subsample = target.reset_index(drop=True).sample(500, replace=False, random_state=state)
        probs_subsample = predictions[target_subsample.index]
        revenues = profit(target_subsample, probs_subsample, 200)
        values.append(revenues)

    values = pd.Series(values)
    mean = values.mean()
    print('Средняя прибыль, руб.: {:,.2f}'.format(mean))
    
    lower = values.quantile(.025)
    upper = values.quantile(.975)
    print('95% доверительный интервал:', '{:,.2f}'.format(lower), ':', '{:,.2f}'.format(upper))

    loss_probability = (values<0).mean()
    print('Вероятность убытков: {:.1%}'.format(loss_probability))
    print()
    
    return mean

#### Регион 1

In [18]:
profit_0 = region_profit(target_valid_0, predictions_0)

Средняя прибыль, руб.: 380,710,890.71
95% доверительный интервал: -126,947,638.03 : 879,613,967.85
Вероятность убытков: 7.2%



#### Регион 2

In [19]:
profit_1 = region_profit(target_valid_1, predictions_1)

Средняя прибыль, руб.: 454,785,434.77
95% доверительный интервал: 46,730,084.77 : 840,213,356.26
Вероятность убытков: 1.3%



#### Регион 3

In [20]:
profit_2 = region_profit(target_valid_2, predictions_2)

Средняя прибыль, руб.: 389,217,073.69
95% доверительный интервал: -115,609,565.78 : 906,512,590.17
Вероятность убытков: 7.3%



## Вывод 

Задача проекта звучала следующим образом: "После оценки рисков нужно оставить те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбрать регион с наибольшей средней прибылью."

Условию вероятности убытков до 2.5% удовлетворяет только Регион 2, у него же наибольшая потенциальная прибыль - 454 млн руб.
По совокупности этих факторов рекомендую для разработки выбрать Регион 2.